# 002 EDA Transforms Workshop

IMPORT DEPENDENCIES

In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

DB MANAGER

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

SQLAlqchemy ENGINE

In [4]:
# Crear el motor de SQLAlchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

LOADING / READING STAGING TABLE

In [5]:
db_table_name = "etl_staging_table_final_workshop"
with engine.connect() as conn:
    df_transform = pd.read_sql(f"SELECT * FROM {db_table_name}", conn)
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,time_ms_ibeta2_full_near_img,time_ms_ibeta2_full_far_img,queue_time_ms_ibeta2_full_near_img,queue_time_ms_ibeta2_full_far_img,time_ms_ibeta2_clip_near_img,time_ms_ibeta2_clip_far_img,queue_time_ms_ibeta2_clip_near_img,queue_time_ms_ibeta2_clip_far_img,time_ms_geometry_check,queue_time_ms_geometry_check
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08T22:10:36.542487-05:00,...,83.0,83.0,6.0,2.0,9.0,9.0,3.0,2.0,17.0,2.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08T22:14:12.653531-05:00,...,40.0,28.0,3.0,1.0,7.0,16.0,2.0,2.0,8.0,1.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08T22:16:10.768124-05:00,...,31.0,58.0,3.0,3.0,9.0,14.0,3.0,2.0,5.0,1.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08T22:16:14.887930-05:00,...,36.0,60.0,2.0,2.0,11.0,10.0,2.0,1.0,23.0,2.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08T22:28:01.054463-05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


EDA GENERAL INFO

In [6]:
print(df_transform.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14654 entries, 0 to 14653
Data columns (total 79 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   api_hash                                     14654 non-null  object 
 1   user_name                                    14654 non-null  object 
 2   file_token                                   14654 non-null  object 
 3   service                                      14654 non-null  object 
 4   summary_code                                 14654 non-null  int64  
 5   summary_status                               14654 non-null  object 
 6   summary_desc                                 14654 non-null  object 
 7   http_code                                    14654 non-null  int64  
 8   total_time_ms                                14654 non-null  int64  
 9   date                                         14654 non-null  object 
 10

In [7]:
print(df_transform.describe())

       summary_code  http_code  total_time_ms  img_size_near_img_width  \
count  14654.000000    14654.0   14654.000000             14654.000000   
mean    1197.237341      200.0    1994.429507               740.515013   
std      583.275790        0.0    1361.465970               280.978414   
min      102.000000      200.0       0.000000               414.000000   
25%     1400.000000      200.0    1207.750000               590.000000   
50%     1400.000000      200.0    1749.000000               590.000000   
75%     1400.000000      200.0    1954.750000               736.000000   
max     3002.000000      200.0    9114.000000              1440.000000   

       img_size_near_img_height  img_size_far_img_width  \
count              14654.000000            14654.000000   
mean                1076.862495              740.624198   
std                  409.914160              281.032593   
min                  480.000000              414.000000   
25%                  896.000000       

In [8]:
for column in df_transform.columns:
    print(f"Column: {column}")
    print(f"Missing Values: {df_transform[column].isnull().sum()} / {len(df_transform)} ({df_transform[column].isnull().sum() / len(df_transform) * 100:.2f}%)")
    # print(f"Unique Values: {df_transform[column].nunique()}")
    print(f"Type Variable: {df_transform[column].dtype}")
    print("-" * 40)

Column: api_hash
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: user_name
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: file_token
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: service
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: summary_code
Missing Values: 0 / 14654 (0.00%)
Type Variable: int64
----------------------------------------
Column: summary_status
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: summary_desc
Missing Values: 0 / 14654 (0.00%)
Type Variable: object
----------------------------------------
Column: http_code
Missing Values: 0 / 14654 (0.00%)
Type Variable: int64
----------------------------------------
Column: total_time_ms
Missing Values: 0 / 14654 (0.00%)
Type Variable: in

MISSING VALUES DUE TO THE MICROSERVICE IS NOT CALCULATED

CLEANING & TRANSFORM FUNCTIONS

In [9]:
import pandas as pd

# Check for missing values and frequency of the most common values
def show_most_frequent_values(df, columns=None, top_n=5):
    """
    Displays the most frequent values in the specified columns of the DataFrame.
    
    Args:
    df (pd.DataFrame): DataFrame containing the columns.
    columns (list, optional): List of column names to check. If None, all columns are checked.
    top_n (int, optional): Number of top frequent values to display. Default is 5.
    
    Returns:
    None
    """
    if columns is None:
        columns = df.columns
    for column in columns:
        try:
            print(f"Column: {column}")
            # Convert column to string if it contains non-1-dimensional data
            if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
                df[column] = df[column].apply(str)
            print(df[column].value_counts().head(top_n))
            print("\n")
        except ValueError as e:
            print(f"Could not process column {column}: {e}")
            print("\n")

# Transform Aspect Ratio Img Size Columns
def transform_imgsize_aspect_ratio(row, width_col, height_col):
    """
    Determines the aspect ratio of an image based on its width and height.
    
    Args:
    row (pd.Series): Row containing the width and height columns.
    width_col (str): Name of the column containing the width.
    height_col (str): Name of the column containing the height.
    
    Returns:
    str: Aspect ratio category ('SQUARE', 'LANDSCAPE', 'PORTRAIT').
    """
    if row[width_col] == row[height_col]:
        return 'SQUARE'
    elif row[width_col] > row[height_col]:
        return 'LANDSCAPE'
    else:
        return 'PORTRAIT'

def transform_img_quality(row, width_col, height_col):
    """
    Determines the quality of an image based on its resolution.
    
    Args:
    row (pd.Series): Row containing the width and height columns.
    width_col (str): Name of the column containing the width.
    height_col (str): Name of the column containing the height.
    
    Returns:
    str: Image quality category ('8K', '4K', 'ULTRAHD', 'FULLHD', 'HD', 'XGA', 'SD', 'QVGA', 'LOWERSCALES').
    """
    resolution = row[width_col] * row[height_col]
    if resolution >= 7680 * 4320:
        return '8K'
    elif resolution >= 4096 * 2160:
        return '4K'
    elif resolution >= 3840 * 2160:
        return 'ULTRAHD'
    elif resolution >= 1920 * 1080:
        return 'FULLHD'
    elif resolution >= 1280 * 720:
        return 'HD'
    elif resolution >= 1024 * 768:
        return 'XGA'
    elif resolution >= 640 * 480:
        return 'SD'
    elif resolution >= 320 * 240:
        return 'QVGA'
    else:
        return 'LOWERSCALES'

# Standardize / Transform Date Columns
def transform_date_by_range(df, date_col):
    """
    Transforms a date column into multiple date-related columns.
    
    Args:
    df (pd.DataFrame): DataFrame containing the date column.
    date_col (str): Name of the date column.
    
    Returns:
    pd.DataFrame: DataFrame with new date-related columns.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    df[date_col+'_year'] = df[date_col].dt.year
    df[date_col+'_month'] = df[date_col].dt.month
    df[date_col+'_day'] = df[date_col].dt.day
    df[date_col+'_hour'] = df[date_col].dt.hour
    df[date_col+'_minute'] = df[date_col].dt.minute
    
    # Mapping month and day
    month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    day_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    
    df[date_col+'_month'] = df[date_col].dt.month.map(month_map)
    df[date_col+'_day_off_week'] = df[date_col].dt.dayofweek.map(day_map)
    df[date_col+'_minute'] = df[date_col].dt.minute
    #df = df.drop(columns=[date_col])
    return df

def sum_columns(row, columns):
    return row[columns].sum()

def subtract_columns(df, col1, col2, new_col_name):
    """
    Subtracts the values of col2 from the values of col1 and adds the result as a new column.
    
    Args:
    df (pd.DataFrame): DataFrame containing the columns.
    col1 (str): Name of the first column.
    col2 (str): Name of the second column.
    new_col_name (str): Name of the new column that will contain the result of the subtraction.
    
    Returns:
    pd.DataFrame: DataFrame with the new column added.
    """
    df[new_col_name] = df[col1] - df[col2]
    return df

def groupby_tail_sorted(df, feature, date_col):
    """
    Sorts the DataFrame by the date column in ascending order, groups by the specified feature, 
    and returns the last row of each group.
    
    Args:
    df (pd.DataFrame): DataFrame to be sorted and grouped.
    feature (str): Name of the column to group by.
    date_col (str): Name of the date column to sort by.
    
    Returns:
    pd.DataFrame: DataFrame containing the last row of each group after sorting.
    """
    # Ensure the DataFrame is sorted by the date column in ascending order
    df = df.sort_values(by=date_col, ascending=True)
    
    # Group by the feature and get the last row of each group
    return df.groupby(feature).tail(1)

def fillna_columns(df, columns, value):
    """
    Fills NaN values in the specified columns with the given value.
    
    Args:
    df (pd.DataFrame): DataFrame containing the columns.
    columns (list): List of column names to fill NaN values.
    value: Value to replace NaN values with.
    
    Returns:
    pd.DataFrame: DataFrame with NaN values filled in the specified columns.
    """
    df[columns] = df[columns].fillna(value)
    return df

def compare_columns(row, col1, col2):
    """
    Compares two columns in a DataFrame row and returns True if they are equal, else False.
    
    Args:
    row (pd.Series): Row containing the columns to compare.
    col1 (str): Name of the first column.
    col2 (str): Name of the second column.
    
    Returns:
    bool: True if the columns are equal, else False.
    """
    return row[col1] == row[col2]

**TRANSFORM DATE COLUMNS BY DATE RANGES**

In [10]:
df_transform = transform_date_by_range(df_transform, 'date')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,queue_time_ms_ibeta2_clip_near_img,queue_time_ms_ibeta2_clip_far_img,time_ms_geometry_check,queue_time_ms_geometry_check,date_year,date_month,date_day,date_hour,date_minute,date_day_off_week
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,3.0,2.0,17.0,2.0,2025,January,8,22,10,Wednesday
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,2.0,2.0,8.0,1.0,2025,January,8,22,14,Wednesday
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,3.0,2.0,5.0,1.0,2025,January,8,22,16,Wednesday
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,2.0,1.0,23.0,2.0,2025,January,8,22,16,Wednesday
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,NaN,NaN,NaN,NaN,2025,January,8,22,28,Wednesday


In [11]:
df_transform[['date_year', 'date_month', 'date_day','date_day_off_week', 'date_hour','date_minute']].tail()

,date_year,date_month,date_day,date_day_off_week,date_hour,date_minute
14649,2025,January,31,Friday,22,34
14650,2025,January,31,Friday,22,35
14651,2025,January,31,Friday,22,35
14652,2025,January,31,Friday,22,53
14653,2025,January,31,Friday,22,53


**TRANSFORM ASPECT RATIO**

In [12]:
df_transform['aspect_ratio_near_img'] = df_transform.apply(transform_imgsize_aspect_ratio, axis=1, width_col='img_size_near_img_width', height_col='img_size_near_img_height')
df_transform['aspect_ratio_far_img'] = df_transform.apply(transform_imgsize_aspect_ratio, axis=1, width_col='img_size_far_img_width', height_col='img_size_far_img_height')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,time_ms_geometry_check,queue_time_ms_geometry_check,date_year,date_month,date_day,date_hour,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,17.0,2.0,2025,January,8,22,10,Wednesday,PORTRAIT,PORTRAIT
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,8.0,1.0,2025,January,8,22,14,Wednesday,PORTRAIT,PORTRAIT
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,5.0,1.0,2025,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,23.0,2.0,2025,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,NaN,NaN,2025,January,8,22,28,Wednesday,LANDSCAPE,LANDSCAPE


**TRANSFORM COMPARE ASPECT RATIO**

In [13]:
df_transform['same_aspect_ratio'] = df_transform.apply(compare_columns, axis=1, col1='aspect_ratio_far_img', col2='aspect_ratio_near_img')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,queue_time_ms_geometry_check,date_year,date_month,date_day,date_hour,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,2.0,2025,January,8,22,10,Wednesday,PORTRAIT,PORTRAIT,True
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,1.0,2025,January,8,22,14,Wednesday,PORTRAIT,PORTRAIT,True
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,1.0,2025,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,2.0,2025,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,NaN,2025,January,8,22,28,Wednesday,LANDSCAPE,LANDSCAPE,True


In [14]:
df_transform[['same_aspect_ratio']].tail()

,same_aspect_ratio
14649,True
14650,True
14651,True
14652,True
14653,True


**TRANSFORM QUALITY IMAGE**

In [15]:
df_transform['image_quality_near_img'] = df_transform.apply(transform_img_quality, axis=1, width_col='img_size_near_img_width', height_col='img_size_near_img_height')
df_transform['image_quality_far_img'] = df_transform.apply(transform_img_quality, axis=1, width_col='img_size_far_img_width', height_col='img_size_near_img_height')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,date_month,date_day,date_hour,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,January,8,22,10,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,January,8,22,14,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,January,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,January,8,22,28,Wednesday,LANDSCAPE,LANDSCAPE,True,SD,SD


In [16]:
df_transform[['image_quality_near_img','image_quality_far_img']].tail()

,image_quality_near_img,image_quality_far_img
14649,SD,SD
14650,SD,SD
14651,SD,SD
14652,SD,SD
14653,SD,SD


**TRANSFORM COMPARE QUALITY IMAGE**

In [17]:
df_transform['same_image_quality'] = df_transform.apply(compare_columns, axis=1, col1='image_quality_far_img', col2='image_quality_near_img')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,date_day,date_hour,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,8,22,10,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,8,22,14,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,8,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,8,22,28,Wednesday,LANDSCAPE,LANDSCAPE,True,SD,SD,True


In [18]:
df_transform[['same_image_quality']].tail()

,same_image_quality
14649,True
14650,True
14651,True
14652,True
14653,True


**TRANSFORM MISSING VALUES RELATED WITH TIME**

In [19]:
# Filter columns related to 'time' execution
columns_to_fill = df_transform.filter(regex='time').columns.tolist()
df_transform = fillna_columns(df_transform, columns_to_fill, value=0)

for column in df_transform[columns_to_fill].columns:
    print(f"Column: {column}")
    print(f"Missing Values: {df_transform[column].isnull().sum()} / {len(df_transform)} ({df_transform[column].isnull().sum() / len(df_transform) * 100:.2f}%)")
    # print(f"Unique Values: {df_transform[column].nunique()}")
    print(f"Type Variable: {df_transform[column].dtype}")
    print("-" * 40)

Column: total_time_ms
Missing Values: 0 / 14654 (0.00%)
Type Variable: int64
----------------------------------------
Column: time_ms_accessory_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_accessory_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_accessory_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_accessory_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_face_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_face_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_face_dete

**TRANSFORM MISSING VALUES RELATED WITH METRICS**

In [20]:
# Filter columns related to 'time' execution
columns_to_fill = df_transform.filter(regex='time').columns.tolist()
df_transform = fillna_columns(df_transform, columns_to_fill, value=0)

for column in df_transform[columns_to_fill].columns:
    print(f"Column: {column}")
    print(f"Missing Values: {df_transform[column].isnull().sum()} / {len(df_transform)} ({df_transform[column].isnull().sum() / len(df_transform) * 100:.2f}%)")
    # print(f"Unique Values: {df_transform[column].nunique()}")
    print(f"Type Variable: {df_transform[column].dtype}")
    print("-" * 40)

Column: total_time_ms
Missing Values: 0 / 14654 (0.00%)
Type Variable: int64
----------------------------------------
Column: time_ms_accessory_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_accessory_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_accessory_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_accessory_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_face_detector_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: time_ms_face_detector_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: queue_time_ms_face_dete

In [21]:
# Filter columns related to 'prob' execution
columns_to_fill = df_transform.filter(regex='prob').columns.tolist()
df_transform = fillna_columns(df_transform, columns_to_fill, value=-1) # -1 means that the value is unknown due to this is probability is not calculated

for column in df_transform[columns_to_fill].columns:
    print(f"Column: {column}")
    print(f"Missing Values: {df_transform[column].isnull().sum()} / {len(df_transform)} ({df_transform[column].isnull().sum() / len(df_transform) * 100:.2f}%)")
    # print(f"Unique Values: {df_transform[column].nunique()}")
    print(f"Type Variable: {df_transform[column].dtype}")
    print("-" * 40)

Column: prob_as_35_selfies_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_as_35_selfies_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_crops_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_crops_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_full_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_full_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_clip_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: prob_ibeta2_clip_far_img
Missing Values: 0 / 14654 (0.00%)
Type V

In [22]:
# Filter columns related to 'score' execution
columns_to_fill = df_transform.filter(regex='score').columns.tolist()
df_transform = fillna_columns(df_transform, columns_to_fill, value=-2) # -2 means that the value is unknown due to this is score is not calculated (this value in order to not affect score metrics)

for column in df_transform[columns_to_fill].columns:
    print(f"Column: {column}")
    print(f"Missing Values: {df_transform[column].isnull().sum()} / {len(df_transform)} ({df_transform[column].isnull().sum() / len(df_transform) * 100:.2f}%)")
    # print(f"Unique Values: {df_transform[column].nunique()}")
    print(f"Type Variable: {df_transform[column].dtype}")
    print("-" * 40)

Column: score_liqe_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_liqe_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_topiq_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_topiq_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_sharpness_classic_metrics_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_sharpness_classic_metrics_far_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_colorfulness_classic_metrics_near_img
Missing Values: 0 / 14654 (0.00%)
Type Variable: float64
----------------------------------------
Column: score_colorfulness_classic_metrics_far_img
Mi

**TRANSFORM GETTING TOTAL TIME BY MICROSERVICE**

In [23]:
# Filter columns related to 'time_ms_accessory_detector' execution
columns_to_sum = df_transform.filter(regex='time_ms_accessory_detector').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_accessory_detector'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_accessory_detector_near_img', 'time_ms_accessory_detector_far_img', 'queue_time_ms_accessory_detector_near_img', 'queue_time_ms_accessory_detector_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,date_hour,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,22,10,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,106.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,22,14,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,103.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,69.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,22,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,67.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,22,28,Wednesday,LANDSCAPE,LANDSCAPE,True,SD,SD,True,85.0


In [24]:
# Filter columns related to 'time_ms_face_detector' execution
columns_to_sum = df_transform.filter(regex='time_ms_face_detector').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_face_detector'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_face_detector_near_img', 'time_ms_face_detector_far_img', 'queue_time_ms_face_detector_near_img', 'queue_time_ms_face_detector_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,date_minute,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,10,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,106.0,119.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,14,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,103.0,99.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,69.0,92.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,16,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,67.0,91.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,28,Wednesday,LANDSCAPE,LANDSCAPE,True,SD,SD,True,85.0,92.0


In [25]:
# Filter columns related to 'time_ms_liqe' execution
columns_to_sum = df_transform.filter(regex='time_ms_liqe').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_liqe'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_liqe_near_img', 'time_ms_liqe_far_img', 'queue_time_ms_liqe_near_img', 'queue_time_ms_liqe_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,date_day_off_week,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,106.0,119.0,373.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,103.0,99.0,238.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,69.0,92.0,376.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,Wednesday,PORTRAIT,PORTRAIT,True,SD,SD,True,67.0,91.0,216.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,Wednesday,LANDSCAPE,LANDSCAPE,True,SD,SD,True,85.0,92.0,36.0


In [26]:
# Filter columns related to 'time_ms_topiq' execution
columns_to_sum = df_transform.filter(regex='time_ms_topiq').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_topiq'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_topiq_near_img', 'time_ms_topiq_far_img', 'queue_time_ms_topiq_near_img', 'queue_time_ms_topiq_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,aspect_ratio_near_img,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,PORTRAIT,PORTRAIT,True,SD,SD,True,106.0,119.0,373.0,616.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,PORTRAIT,PORTRAIT,True,SD,SD,True,103.0,99.0,238.0,456.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,PORTRAIT,PORTRAIT,True,SD,SD,True,69.0,92.0,376.0,373.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,PORTRAIT,PORTRAIT,True,SD,SD,True,67.0,91.0,216.0,334.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,LANDSCAPE,LANDSCAPE,True,SD,SD,True,85.0,92.0,36.0,171.0


In [27]:
# Filter columns related to 'time_ms_classic_metrics' execution
columns_to_sum = df_transform.filter(regex='time_ms_classic_metrics').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_classic_metrics'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_classic_metrics_near_img', 'time_ms_classic_metrics_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,aspect_ratio_far_img,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,PORTRAIT,True,SD,SD,True,106.0,119.0,373.0,616.0,614.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,PORTRAIT,True,SD,SD,True,103.0,99.0,238.0,456.0,286.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,PORTRAIT,True,SD,SD,True,69.0,92.0,376.0,373.0,388.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,PORTRAIT,True,SD,SD,True,67.0,91.0,216.0,334.0,274.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,LANDSCAPE,True,SD,SD,True,85.0,92.0,36.0,171.0,91.0


In [28]:
# Filter columns related to 'time_ms_as_35_selfies' execution
columns_to_sum = df_transform.filter(regex='time_ms_as_35_selfies').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_as_35_selfies'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_as_35_selfies_near_img', 'time_ms_as_35_selfies_far_img', 'queue_time_ms_as_35_selfies_near_img', 'queue_time_ms_as_35_selfies_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,same_aspect_ratio,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,True,SD,SD,True,106.0,119.0,373.0,616.0,614.0,191.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,True,SD,SD,True,103.0,99.0,238.0,456.0,286.0,85.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,True,SD,SD,True,69.0,92.0,376.0,373.0,388.0,137.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,True,SD,SD,True,67.0,91.0,216.0,334.0,274.0,118.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,True,SD,SD,True,85.0,92.0,36.0,171.0,91.0,0.0


In [29]:
# Filter columns related to 'time_ms_ibeta2_crops' execution
columns_to_sum = df_transform.filter(regex='time_ms_ibeta2_crops').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_ibeta2_crops'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_ibeta2_crops_near_img', 'time_ms_ibeta2_crops_far_img', 'queue_time_ms_ibeta2_crops_near_img', 'queue_time_ms_ibeta2_crops_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,image_quality_near_img,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,SD,SD,True,106.0,119.0,373.0,616.0,614.0,191.0,157.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,SD,SD,True,103.0,99.0,238.0,456.0,286.0,85.0,85.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,SD,SD,True,69.0,92.0,376.0,373.0,388.0,137.0,129.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,SD,SD,True,67.0,91.0,216.0,334.0,274.0,118.0,118.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,SD,SD,True,85.0,92.0,36.0,171.0,91.0,0.0,0.0


In [30]:
# Filter columns related to 'time_ms_ibeta2_full' execution
columns_to_sum = df_transform.filter(regex='time_ms_ibeta2_full').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_ibeta2_full'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_ibeta2_full_near_img', 'time_ms_ibeta2_full_far_img', 'queue_time_ms_ibeta2_full_near_img', 'queue_time_ms_ibeta2_full_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,image_quality_far_img,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,SD,True,106.0,119.0,373.0,616.0,614.0,191.0,157.0,174.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,SD,True,103.0,99.0,238.0,456.0,286.0,85.0,85.0,72.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,SD,True,69.0,92.0,376.0,373.0,388.0,137.0,129.0,95.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,SD,True,67.0,91.0,216.0,334.0,274.0,118.0,118.0,100.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,SD,True,85.0,92.0,36.0,171.0,91.0,0.0,0.0,0.0


In [31]:
# Filter columns related to 'time_ms_ibeta2_clip' execution
columns_to_sum = df_transform.filter(regex='time_ms_ibeta2_clip').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_ibeta2_clip'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_ibeta2_clip_near_img', 'time_ms_ibeta2_clip_far_img', 'queue_time_ms_ibeta2_clip_near_img', 'queue_time_ms_ibeta2_clip_far_img']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,same_image_quality,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,True,106.0,119.0,373.0,616.0,614.0,191.0,157.0,174.0,23.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,True,103.0,99.0,238.0,456.0,286.0,85.0,85.0,72.0,27.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,True,69.0,92.0,376.0,373.0,388.0,137.0,129.0,95.0,28.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,True,67.0,91.0,216.0,334.0,274.0,118.0,118.0,100.0,24.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,True,85.0,92.0,36.0,171.0,91.0,0.0,0.0,0.0,0.0


In [32]:
# Filter columns related to 'time_ms_geometry_check' execution
columns_to_sum = df_transform.filter(regex='time_ms_geometry_check').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_geometry_check'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
#Dropping columns to sum
df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['time_ms_geometry_check', 'queue_time_ms_geometry_check']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,total_time_ms_accessory_detector,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip,total_time_ms_geometry_check
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,106.0,119.0,373.0,616.0,614.0,191.0,157.0,174.0,23.0,19.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,103.0,99.0,238.0,456.0,286.0,85.0,85.0,72.0,27.0,9.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,69.0,92.0,376.0,373.0,388.0,137.0,129.0,95.0,28.0,6.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,67.0,91.0,216.0,334.0,274.0,118.0,118.0,100.0,24.0,25.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,85.0,92.0,36.0,171.0,91.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Filter columns related to 'total_time_ms' execution
columns_to_sum = df_transform.filter(regex='total_time_ms_').columns.tolist()
print(columns_to_sum)
df_transform['total_time_ms_all_mircroservices'] = df_transform.apply(lambda row: sum_columns(row, columns_to_sum), axis=1)
# #Dropping columns to sum
# df_transform = df_transform.drop(columns=columns_to_sum)
df_transform.head()

['total_time_ms_accessory_detector', 'total_time_ms_face_detector', 'total_time_ms_liqe', 'total_time_ms_topiq', 'total_time_ms_classic_metrics', 'total_time_ms_as_35_selfies', 'total_time_ms_ibeta2_crops', 'total_time_ms_ibeta2_full', 'total_time_ms_ibeta2_clip', 'total_time_ms_geometry_check']


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,total_time_ms_face_detector,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip,total_time_ms_geometry_check,total_time_ms_all_mircroservices
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,119.0,373.0,616.0,614.0,191.0,157.0,174.0,23.0,19.0,2392.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,99.0,238.0,456.0,286.0,85.0,85.0,72.0,27.0,9.0,1460.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,92.0,376.0,373.0,388.0,137.0,129.0,95.0,28.0,6.0,1693.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,91.0,216.0,334.0,274.0,118.0,118.0,100.0,24.0,25.0,1367.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,92.0,36.0,171.0,91.0,0.0,0.0,0.0,0.0,0.0,475.0


**TRANSFORM GETTING OTHERTIME RESPONSE FROM OVERALL**

In [34]:
df_transform = subtract_columns(df_transform, 'total_time_ms', 'total_time_ms_all_mircroservices', 'total_time_ms_other_mircroservices')
df_transform.head()

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,total_time_ms_liqe,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip,total_time_ms_geometry_check,total_time_ms_all_mircroservices,total_time_ms_other_mircroservices
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08 22:10:36.542487-05:00,...,373.0,616.0,614.0,191.0,157.0,174.0,23.0,19.0,2392.0,-188.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08 22:14:12.653531-05:00,...,238.0,456.0,286.0,85.0,85.0,72.0,27.0,9.0,1460.0,377.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08 22:16:10.768124-05:00,...,376.0,373.0,388.0,137.0,129.0,95.0,28.0,6.0,1693.0,243.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08 22:16:14.887930-05:00,...,216.0,334.0,274.0,118.0,118.0,100.0,24.0,25.0,1367.0,273.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08 22:28:01.054463-05:00,...,36.0,171.0,91.0,0.0,0.0,0.0,0.0,0.0,475.0,444.0


**REORDER TRANSFORM DATAFRAME**

In [35]:
column_order = ['api_hash', 'user_name', 'file_token', 'service', 'summary_code',
       'summary_status', 'summary_desc', 'http_code',
       'procesoConvenioGuid', 'convenio', 'documento',
       'img_size_near_img_width', 'img_size_near_img_height',
       'img_size_far_img_width', 'img_size_far_img_height','image_quality_near_img', 'image_quality_far_img','same_image_quality',
       'aspect_ratio_near_img', 'aspect_ratio_far_img','same_aspect_ratio',
       'score_liqe_near_img', 'score_liqe_far_img', 'score_topiq_near_img',
       'score_topiq_far_img', 'score_sharpness_classic_metrics_near_img',
       'score_sharpness_classic_metrics_far_img',
       'score_colorfulness_classic_metrics_near_img',
       'score_colorfulness_classic_metrics_far_img',
       'score_contrast_classic_metrics_near_img',
       'score_contrast_classic_metrics_far_img',
       'score_brightness_classic_metrics_near_img',
       'score_brightness_classic_metrics_far_img',
       'score_blur_score_classic_metrics_near_img',
       'score_blur_score_classic_metrics_far_img',
       'score_svd_score_classic_metrics_near_img',
       'score_svd_score_classic_metrics_far_img',
       'prob_as_35_selfies_near_img', 'prob_as_35_selfies_far_img',
       'prob_ibeta2_crops_near_img', 'prob_ibeta2_crops_far_img',
       'prob_ibeta2_full_near_img', 'prob_ibeta2_full_far_img',
       'prob_ibeta2_clip_near_img', 'prob_ibeta2_clip_far_img',
       'prob_as_heuristics_near_img', 'prob_as_heuristics_far_img','date',
       'date_year', 'date_month', 'date_day','date_day_off_week', 'date_hour', 'date_minute',
        'total_time_ms_accessory_detector',
       'total_time_ms_face_detector', 'total_time_ms_liqe',
       'total_time_ms_topiq', 'total_time_ms_classic_metrics',
       'total_time_ms_as_35_selfies', 'total_time_ms_ibeta2_crops',
       'total_time_ms_ibeta2_full', 'total_time_ms_ibeta2_clip',
       'total_time_ms_geometry_check', 'total_time_ms_all_mircroservices',
       'total_time_ms_other_mircroservices', 'total_time_ms']
df_transform = df_transform.reindex(columns=column_order)
print(f"DataFrame Transform with retries len: {len(df_transform)}")
df_transform.head()

DataFrame Transform with retries len: 14654


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,procesoConvenioGuid,convenio,...,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip,total_time_ms_geometry_check,total_time_ms_all_mircroservices,total_time_ms_other_mircroservices,total_time_ms
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_1,convenio_1,...,616.0,614.0,191.0,157.0,174.0,23.0,19.0,2392.0,-188.0,2204
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_2,convenio_1,...,456.0,286.0,85.0,85.0,72.0,27.0,9.0,1460.0,377.0,1837
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_2,convenio_1,...,373.0,388.0,137.0,129.0,95.0,28.0,6.0,1693.0,243.0,1936
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_2,convenio_1,...,334.0,274.0,118.0,118.0,100.0,24.0,25.0,1367.0,273.0,1640
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,procesoConvenioGuid_3,convenio_1,...,171.0,91.0,0.0,0.0,0.0,0.0,0.0,475.0,444.0,919


**TRANSFORM GETTING TRANSACTIONS WITHOUT RETRIES**

In [36]:
df_transform_no_retrys = groupby_tail_sorted(df_transform, "procesoConvenioGuid", "date")
print(f"DataFrame Transform with no retries len: {len(df_transform_no_retrys)}")
df_transform_no_retrys.head()

DataFrame Transform with no retries len: 4427


,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,procesoConvenioGuid,convenio,...,total_time_ms_topiq,total_time_ms_classic_metrics,total_time_ms_as_35_selfies,total_time_ms_ibeta2_crops,total_time_ms_ibeta2_full,total_time_ms_ibeta2_clip,total_time_ms_geometry_check,total_time_ms_all_mircroservices,total_time_ms_other_mircroservices,total_time_ms
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_1,convenio_1,...,616.0,614.0,191.0,157.0,174.0,23.0,19.0,2392.0,-188.0,2204
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,procesoConvenioGuid_2,convenio_1,...,334.0,274.0,118.0,118.0,100.0,24.0,25.0,1367.0,273.0,1640
6,02bc5e7,user_name_1,sWKm2qha-A,main,302,error,blur/haze,200,procesoConvenioGuid_3,convenio_1,...,157.0,75.0,0.0,0.0,0.0,0.0,0.0,470.0,229.0,699
7,02bc5e7,user_name_1,XIZzVUa7iQ,main,406,error,non-frontal,200,procesoConvenioGuid_4,convenio_1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.0,531.0,692
8,02bc5e7,user_name_1,ztDiYSlG4g,main,302,error,blur/haze,200,procesoConvenioGuid_5,convenio_1,...,176.0,123.0,0.0,0.0,0.0,0.0,0.0,560.0,365.0,925


**SAVING TRANSFORM DATA**

In [37]:
df_transform_no_retrys.to_csv("../data/processed/transform_data_no_retries.csv", index=False)
df_transform.to_csv("../data/processed/transform_data.csv", index=False)

The EDA report can be obtained by executing 002_eda_report.ipynb